<a href="https://colab.research.google.com/github/sahilfaizal01/PyTorch-Implementation/blob/main/Convolutional_Neural_Network_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
# for parameterless functions, like some activation functions
import torch.nn.functional as F
# for accessing standard datasets
import torchvision.datasets as datasets
# for performing transformations on the dataset for augmentation
import torchvision.transforms as transforms
# for optimizers like SGD, Adam
from torch import optim
# for all neural network modules
from torch import nn
# for easy data management
from torch.utils.data import (
    DataLoader,
)
# for progress bar
from tqdm import tqdm

In [2]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [3]:
# setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 3e-4 # karpathy's constant
batch_size = 64
num_epochs = 3

In [4]:
# load data
train_dataset = datasets.MNIST(
    root = "dataset/", train = True, transform = transforms.ToTensor(), download = True
)

test_dataset = datasets.MNIST(
    root = "dataset/", train = False, transform = transforms.ToTensor(), download = True
)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 87467782.63it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 27227622.80it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden




100%|██████████| 1648877/1648877 [00:00<00:00, 30980457.26it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4278133.57it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [8]:
# Initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train Network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or adam step
    optimizer.step()

100%|██████████| 938/938 [00:19<00:00, 47.88it/s]


In [13]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    """
    Check accuracy of our trained model given a loader and a model

    Parameters:
        loader: torch.utils.data.DataLoader
            A loader for the dataset you want to check accuracy on
        model: nn.Module
            The model you want to check accuracy on

    Returns:
        acc: float
            The accuracy of the model on the dataset given by the loader
    """

    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
      # Loop through the data
      for x, y in loader:
        # Move data to device
        x = x.to(device=device)
        y = y.to(device=device)
        # # Get to correct shape
        # x = x.reshape(x.shape[0], -1)
        # Forward pass
        scores = model(x)
        _, predictions = scores.max(1) # index of max value
        # Check how many we got correct
        num_correct += (predictions == y).sum()
        # Keep track of number of samples
        num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [14]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 96.22
Accuracy on test set: 96.41


In [15]:
# if loader.dataset.train